In [1]:
import rasterio
import numpy as np

In [2]:
VIIRS_INPUT = r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"

In [3]:
n_bands = 17544   # hourly for 2015–2016
with rasterio.open(VIIRS_INPUT) as src:
    height, width = src.height, src.width
    src_data = src.read(1)  
    profile = src.profile
    profile.update(
        count=n_bands,
        dtype="uint8",
        driver="GTiff"
    )

    with rasterio.open(OUT_FILE, "w", **profile) as dst:
        for i in range(n_bands):
           
            dst.write(src_data.astype("uint8"), i + 1)

In [4]:
print(f"✅ Created VIIRS time stack with {n_bands} bands → {OUT_FILE}")

✅ Created VIIRS time stack with 17544 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif


In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset(r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc")

In [3]:
ds.data_vars

Data variables:
    fire_label  (time, latitude, longitude) uint8 4MB ...

In [5]:
import numpy as np
np.unique(ds['fire_label'])

array([0, 1], dtype=uint8)

In [6]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# Input NetCDF and output GeoTIFF
VIIRS_INPUT = r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack1.tif"

# Open NetCDF with xarray
ds = xr.open_dataset(VIIRS_INPUT)
fire_label = ds["fire_label"]  # (time, lat, lon)

# Dimensions
n_bands = fire_label.sizes["time"]
height = fire_label.sizes["latitude"]
width = fire_label.sizes["longitude"]

print(f"fire_label shape: {fire_label.shape} → (time, lat, lon)")
print(f"Bands (time steps): {n_bands}, Height: {height}, Width: {width}")

# Build rasterio profile
# Assuming lat/lon are regular grids
lat = fire_label.latitude.values
lon = fire_label.longitude.values
res_lat = abs(lat[1] - lat[0])
res_lon = abs(lon[1] - lon[0])

transform = from_origin(lon.min(), lat.max(), res_lon, res_lat)

profile = {
    "driver": "GTiff",
    "height": height,
    "width": width,
    "count": n_bands,
    "dtype": "uint8",
    "crs": "EPSG:4326",  # NetCDF lat/lon is usually WGS84
    "transform": transform
}

# Write GeoTIFF
with rasterio.open(OUT_FILE, "w", **profile) as dst:
    for i in range(n_bands):
        arr = fire_label.isel(time=i).values.astype("uint8")
        dst.write(arr, i + 1)

    # Add metadata for variable
    dst.update_tags(variable="fire_label", description="Binary fire mask (0/1)")

print(f"✅ Created VIIRS time stack with {n_bands} bands → {OUT_FILE}")


fire_label shape: (17544, 13, 17) → (time, lat, lon)
Bands (time steps): 17544, Height: 13, Width: 17
✅ Created VIIRS time stack with 17544 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack1.tif


In [2]:
import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from datetime import datetime

def create_multi_band_frp_raster(csv_path, output_tif_path, width=17, height=13):
    df = pd.read_csv(csv_path)
    
    # Combine date and time for a single datetime object
    df['acq_datetime'] = pd.to_datetime(df['acq_date'] + ' ' + df['acq_time'].astype(str).str.zfill(4), format='%d-%m-%Y %H%M')
    
    # Sort the data by datetime and group by unique hour
    df = df.sort_values('acq_datetime')
    time_groups = df.groupby(pd.Grouper(key='acq_datetime', freq='H'))
    
    # Determine geographic bounds (assuming your data is contained within this area)
    minx, miny, maxx, maxy = (
        df['longitude'].min(),
        df['latitude'].min(),
        df['longitude'].max(),
        df['latitude'].max()
    )
    
    # Calculate the transform for the fixed grid dimensions
    transform = from_bounds(minx, miny, maxx, maxy, width=width, height=height)

    # List to hold the NumPy arrays for each time step (band)
    raster_bands = []
    
    # Process each unique time step
    for time_stamp, group in time_groups:
        # Create an empty raster grid for this hour
        nodata_value = -9999
        hour_raster = np.full((height, width), nodata_value, dtype=np.float32)

        # Iterate through the fire points in this hour's group
        for _, row in group.iterrows():
            lon = row['longitude']
            lat = row['latitude']
            frp = row['frp']
            
            # Convert lat/lon to pixel coordinates
            row_idx, col_idx = rasterio.transform.rowcol(transform, lon, lat)
            
            # Place the FRP value at the correct pixel
            if 0 <= row_idx < height and 0 <= col_idx < width:
                hour_raster[row_idx, col_idx] = frp
        
        raster_bands.append(hour_raster)

    # Stack all the hourly rasters into a single 3D array
    if not raster_bands:
        print("No fire data found in the CSV to create raster bands.")
        return
        
    stacked_raster = np.stack(raster_bands)
    
    # Define the raster's profile for writing
    profile = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': len(raster_bands),  # Number of bands equals the number of hours
        'dtype': np.float32,
        'crs': 'EPSG:4326',  # WGS 84
        'transform': transform,
        'nodata': nodata_value
    }
    
    # Write the 3D array to a multi-band GeoTIFF file
    with rasterio.open(output_tif_path, 'w', **profile) as dst:
        for i, band in enumerate(stacked_raster, 1):
            dst.write(band, i)
    
    print(f"✅ Created VIIRS time stack with {len(raster_bands)} bands → {output_tif_path}")

# Run the function
csv_file = r"C:\Users\Ankit\Datasets_Forest_fire\filtered_viirs.csv"
output_file = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack_frp.tif"
create_multi_band_frp_raster(csv_file, output_file)

C:\Users\Ankit\AppData\Local\Temp\ipykernel_16060\1491515261.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_groups = df.groupby(pd.Grouper(key='acq_datetime', freq='H'))


✅ Created VIIRS time stack with 17521 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack_frp.tif
